In [15]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
# import higherOrderKME
# from higherOrderKME import sigkernel
from src.path_characteristic_function import char_func_path
from src.datasets.data_preparation import prepare_dl
from src.train_regressor import train_regressor, plot_reg_losses
from src.model import LSTMRegressor, LSTMGenerator, LSTMRegressor_
from src.utils import AddTime, to_numpy, track_gradient_norms, track_norm, construct_past_dev_path, construct_future_dev_path
from torch.utils.data import DataLoader, Dataset
import ml_collections
from torch.distributions import Bernoulli
import seaborn as sns
import os
from fbm import FBM
from os import path as pt
import yaml
import ml_collections
from torch import nn
import pickle
import random
from sklearn.model_selection import train_test_split
from collections import defaultdict
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
sns.set()
torch.manual_seed(0)
device = 'cuda'

In [2]:
config_dir = pt.join("configs/configs_DR.yaml")
with open(config_dir) as file:
    config = ml_collections.ConfigDict(yaml.safe_load(file))

In [3]:
data = pickle.load(open('./examples/DR/data_optimal_stopping.obj','rb'))
prices, paths = data['prices'], data['paths']

# data = torch.load('./examples/Rough/train_X.pt')
N, L, D = paths[0].shape
paths_torch = np.stack(paths, axis = 0)

In [4]:
paths_train = torch.tensor(paths_torch[:,:-100]).to(dtype=torch.float, device=device)
paths_test = torch.tensor(paths_torch[:,-100:]).to(dtype=torch.float, device=device)

config.R_input_dim = paths_train.shape[-1]+1
config.data_feat_dim = paths_train.shape[-1]
config.n_lags = paths_train.shape[2]

rank_1_pcf = char_func_path(num_samples=config.Rank_1_num_samples,
                           hidden_size=config.Rank_1_lie_degree,
                           input_dim=paths_train.shape[-1],
                           add_time=True,
                           include_initial=False,
                           return_sequence=False).to(config.device)

rank_1_pcf.load_state_dict(torch.load('./examples/DR/rank_1_pcf.pt'))

<All keys matched successfully>

In [5]:
past_dev_path_trains = []
past_dev_path_tests = []
for i in range(paths_train.shape[0]):
    past_dev_path_train = construct_past_dev_path(rank_1_pcf, AddTime(paths_train[i]).to(device), L)
    past_dev_path_test = construct_past_dev_path(rank_1_pcf, AddTime(paths_test[i]).to(device), L)
#     print(past_dev_path_train.shape)
    past_dev_path_trains.append(past_dev_path_train)
    past_dev_path_tests.append(past_dev_path_test)
    
past_dev_path_trains = torch.stack(past_dev_path_trains)
past_dev_path_tests = torch.stack(past_dev_path_tests)

In [6]:
regressors = []
for i in range(paths_train.shape[0]):
    regressor = LSTMRegressor(
            input_dim=config.R_input_dim,
            hidden_dim=config.R_hidden_dim,
            output_dim=config.R_output_dim,
            n_layers=config.R_num_layers
        )
    regressor.load_state_dict(torch.load('./examples/DR/regression_{}.pt'.format(i)))
    regressor.to(config.device)
    regressors.append(regressor)

In [21]:
class expected_dev():

    def __init__(self, regressors, lie_degree_1, lie_degree_2, num_samples_2, add_time=True, device='cuda', whole_dev=True):
        super(expected_dev, self).__init__()
        """ Generator base class. All generators should be children of this class. """
        self.device = device
        self.regressors = regressors
        
        for regressor in self.regressors:
            regressor.to(device)
            regressor.eval()

        self.lie_degree_1 = lie_degree_1
        self.add_time = add_time
        self.num_samples_2 = num_samples_2
        self.lie_degree_2 = lie_degree_2
        self.pcf_level_2s = defaultdict(list)
        
        for i in range(24):
            for j in range(i,24):
                pcf_level_2 = char_func_path(num_samples=self.num_samples_2,
                                              hidden_size=self.lie_degree_2,
                                              input_dim=2 * self.lie_degree_1 ** 2,
                                              add_time=add_time,
                                              include_initial=False,
                                              return_sequence=False)
                pcf_level_2.to(device)
                self.pcf_level_2s['{},{}'.format(i,j)] = pcf_level_2
        for v in self.pcf_level_2s.values():
#             print(v)
            v.load_state_dict(self.pcf_level_2s['0,1'].state_dict())
            
        self.whole_dev = whole_dev
        
def get_gram_matrix(expected_devx, paths_A, past_dev_path_A, ind_A, paths_B, past_dev_path_B, ind_B):
    
    N1, M, L, D = paths_A.shape
    N2, M, L, D = paths_B.shape

    gram_matrix = torch.zeros(N1,N2)
    for i in range(N1):
        for j in range(i,N2):
            expected_devx.pcf_level_2s['{},{}'.format(i,j)].eval()
            X = paths_A[i]
            Y = paths_B[j]
            
#             X, past_dev_X = next(iter(paths_train[i]))
#             Y, past_dev_Y = next(iter(paths_train[j]))
            
#             M, L, D = X.shape
            
            past_dev_X = past_dev_path_A[i]
            past_dev_Y = past_dev_path_B[j]
            with torch.no_grad():
                if expected_devx.whole_dev:
                    exp_dev_X = expected_devx.regressors[ind_A[i]](AddTime(X), expected_devx.device)
                    exp_dev_Y = expected_devx.regressors[ind_B[j]](AddTime(Y), expected_devx.device)
                    exp_dev_X = (past_dev_X @ exp_dev_X).reshape([-1, X.shape[1], expected_devx.lie_degree_1 ** 2])
                    exp_dev_Y = (past_dev_Y @ exp_dev_Y).reshape([-1, X.shape[1], expected_devx.lie_degree_1 ** 2])
                else:
                    exp_dev_X = expected_devx.regressors[ind_A[i]](AddTime(X), expected_devx.device).reshape([-1, X.shape[1], expected_devx.lie_degree_1 ** 2])
                    exp_dev_Y = expected_devx.regressors[ind_B[j]](AddTime(Y), expected_devx.device).reshape([-1, X.shape[1], expected_devx.lie_degree_1 ** 2])

                exp_dev_X = torch.cat([exp_dev_X.real, exp_dev_X.imag], -1)
                exp_dev_Y = torch.cat([exp_dev_Y.real, exp_dev_Y.imag], -1)
                
                gram_matrix[i,j] = expected_devx.pcf_level_2s['{},{}'.format(i,j)].distance_measure(exp_dev_X, exp_dev_Y, Lambda=0)
    return gram_matrix

class option_regressor(nn.Module):
    def __init__(self, input_dim):
        super(option_regressor, self).__init__()
        self.fc1 = nn.Linear(input_dim, input_dim)  # One input feature, 64 hidden units
        self.fc2 = nn.Linear(input_dim, 1)   # 64 hidden units, one output feature
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    
def train_option_regressor(gram_matrix, prices, gram_matrix_test, prices_test, num_epochs):
    torch.manual_seed(0)
    # Instantiate the model
    model = option_regressor(gram_matrix.shape[1]).to(device)

    # Define loss function and optimizer
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    losses = []
    criterion = nn.MSELoss()
    for epoch in tqdm(range(num_epochs)):
        # Forward pass
        outputs = model(gram_matrix.to(device))
#         loss = ridge_loss(outputs, prices, model, 1)
        loss = criterion(outputs, prices)
        losses.append(loss.item())
#         print(loss.item())
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if epoch% 50 == 0:
            model.eval()
            criterion = nn.MSELoss()
            with torch.no_grad():
                pred = model(gram_matrix_test.to(device))
                test_loss = criterion(pred, prices_test)
#                 print('epoch: ', epoch, 'train_loss: ', loss.item())
#                 print('epoch: ', epoch, 'test_loss: ', test_loss.item())
#                 print('y_pred: ', pred.T, 'y_test: ', prices_test.T)
            model.train()
        
    return model, losses, test_loss

In [10]:
prices_torch = torch.tensor(np.array(prices['price'])).to(torch.float).to(device).unsqueeze(0)
prices_torch = prices_torch.t() 

In [11]:
ind_train, ind_test, y_train, y_test = train_test_split(np.arange(len(prices_torch)), 
                                                        to_numpy(prices_torch), 
                                                        test_size=0.1,random_state=1)

In [25]:
test_losses = []

for seed in range(3):
    torch.manual_seed(seed)
    expected_devx = expected_dev(regressors = regressors, 
                                 lie_degree_1 = 5, lie_degree_2 = 8, num_samples_2 = 100, whole_dev=True)
    
    gram_matrix_whole = get_gram_matrix(expected_devx, 
                                    paths_train, 
                                    past_dev_path_trains, 
                                    list(range(N)), 
                                    paths_train, 
                                    past_dev_path_trains, 
                                    list(range(N)))
    gram_matrix_whole_ = gram_matrix_whole + gram_matrix_whole.T
    
    gram_matrix_train = gram_matrix_whole_[ind_train][:,ind_train]
    gram_matrix_test = gram_matrix_whole_[ind_test][:,ind_train]

    model, loss, test_loss = train_option_regressor(torch.exp(-2*gram_matrix_train), 
                                                    torch.tensor(y_train).to(device), 
                                                    torch.exp(-2*gram_matrix_test), 
                                                    torch.tensor(y_test).to(device), 
                                                    1000)
    
    test_losses.append(test_loss)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [29]:
torch.stack(test_losses).mean().item()

0.00035091652534902096

In [31]:
torch.stack(test_losses).std().item()

2.2990907382336445e-05